Driver distraction classification using Convolutional Neural Networks

In [28]:
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator

ModuleNotFoundError: No module named 'matplotlib'

In [27]:
#Upload Training Data
workingdir = os.path.abspath('')
print(workingdir)
c0directory = os.path.join(workingdir + '/state-farm-distracted-driver-detection/imgs/train/c0')
c1directory = os.path.join(workingdir + '/state-farm-distracted-driver-detection/imgs/train/c1')
c2directory = os.path.join(workingdir + '/state-farm-distracted-driver-detection/imgs/train/c2')
c3directory = os.path.join(workingdir + '/state-farm-distracted-driver-detection/imgs/train/c3')
c4directory = os.path.join(workingdir + '/state-farm-distracted-driver-detection/imgs/train/c4')
c5directory = os.path.join(workingdir + '/state-farm-distracted-driver-detection/imgs/train/c5')
c6directory = os.path.join(workingdir + '/state-farm-distracted-driver-detection/imgs/train/c6')
c7directory = os.path.join(workingdir + '/state-farm-distracted-driver-detection/imgs/train/c7')
c8directory = os.path.join(workingdir + '/state-farm-distracted-driver-detection/imgs/train/c8')
c9directory = os.path.join(workingdir + '/state-farm-distracted-driver-detection/imgs/train/c9')
print('total training c0 images:', len(os.listdir(c0directory)))
print('total training c0 images:', len(os.listdir(c0directory)))
print('total training c1 images:', len(os.listdir(c1directory)))
print('total training c2 images:', len(os.listdir(c2directory)))
print('total training c3 images:', len(os.listdir(c3directory)))
print('total training c4 images:', len(os.listdir(c4directory)))
print('total training c5 images:', len(os.listdir(c5directory)))
print('total training c6 images:', len(os.listdir(c6directory)))
print('total training c7 images:', len(os.listdir(c7directory)))
print('total training c8 images:', len(os.listdir(c8directory)))
print('total training c9 images:', len(os.listdir(c9directory)))

C:\Users\thami\Documents\GitHub\MachineLearningDriverdistraction
total training c0 images: 2489
total training c0 images: 2489
total training c1 images: 2267
total training c2 images: 2317
total training c3 images: 2346
total training c4 images: 2326
total training c5 images: 2312
total training c6 images: 2325
total training c7 images: 2002
total training c8 images: 1911
total training c9 images: 2129
